In [21]:
import csv
import time
import openai
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)
from langchain.vectorstores import DeepLake
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import json

In [32]:
OPENAI_API_KEY= ""
openai.api_key = OPENAI_API_KEY
SLEEP_TIME=2
INPUT_PATH="input\\test.csv"
OUTPUT_PATH="output\\output.csv"
PROMPT="Summarize the conversation with detailed information and speaker using point form:   "
CONTEXT="You are a helpful assistant for summarizing a meeting transcript."
TRANSCRIPT_PATH="trans\\chat_session.pdf"

In [33]:
# load document
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(TRANSCRIPT_PATH)
documents = loader.load()

In [34]:
# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)


In [35]:
def call_endpoint(input_csv_path, output_csv_path, prompt):
    with open(input_csv_path, 'r') as input_file, open(output_csv_path, 'w', newline='') as output_file:
        reader = csv.reader(input_file)
        writer = csv.writer(output_file)
        for row in reader:
            p = prompt  + " : " + row[0]
            print(p)
            # messages = [
            #     {"role": "system", "content": CONTEXT},
            #     {"role": "user", "content": p}
            # ]
            # completion = openai.ChatCompletion.create(
            #     model="gpt-3.5-turbo",
            #     messages=messages
            # )
            # time.sleep(SLEEP_TIME)
            # output_text = completion.choices[0].message.content
            result = qa({"query": p})
            output_text= result['result']
            writer.writerow([output_text])


call_endpoint(INPUT_PATH,OUTPUT_PATH, prompt=PROMPT)

Summarize the conversation with detailed information and speaker using point form:    : Certain regional offices (Singapore, Japan, Cambodia, Philipines, China) are currently adopting different change management systems, but they are in the process of migrating to ServiceNow to establish standardized control and processes.
Summarize the conversation with detailed information and speaker using point form:    : The current change management process lacks a formal communication plan to adequately inform stakeholders and business owners about change implementations.
Summarize the conversation with detailed information and speaker using point form:    : Currently, there is a lack of regular reviews of implemented production changes to verify adherence to approval, testing, and review procedures. 
Summarize the conversation with detailed information and speaker using point form:    : The current change management process lacks clarity in the process for conducting risk and impact assessments